<a href="https://colab.research.google.com/github/Annu1782003/task4/blob/main/Copy_of_Untitled14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d asaniczka/reddit-on-israel-palestine-daily-updated

 99% 462M/466M [00:08<00:00, 85.9MB/s]
100% 466M/466M [00:08<00:00, 56.0MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/reddit-on-israel-palestine-daily-updated.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
reddit_data = pd.read_csv('/content/reddit_opinion_PSE_ISR.csv', encoding = 'ISO-8859-1')

<ipython-input-9-e06255c920f9>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  reddit_data = pd.read_csv('/content/reddit_opinion_PSE_ISR.csv', encoding = 'ISO-8859-1')


In [ ]:
reddit_data.shape

(1138801, 24)

In [ ]:
reddit_data.head()

,,,,,,,,,,,,,,comment id,score,self text,subreddit,created time,post id,author name,controversialty,ups,downs
comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,user_is_verified,user_account_created_time,user_awardee_karma,user_awarder_karma,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
kyctini,1,"Youch... if the pushback services are provided directly by Heathrow, then that's their error and their expense.",worldnews,2024-04-06 18:18:49,1bxi4d6,Ehldas,0,1,0,True,2014-04-22 23:13:18,4000.0,0.0,1587.0,444662.0,450249.0,4,NaN,Two planes at Heathrow Airport collide on the ...,1.0,4,0,2024-04-06 18:11:21
kycth3i,1,"Do you think if turks where the victims arabs wouldn't care do you think they dont care about turkey ? the earth quake millions donated , the lira fall everyone went to turkey for tourism to support the economy , whenever we see a turkish product vs the chinese one we buy the turkish for the massacre in xinjang the uyghurs , the karabagh war everyone cheered for azerbaijan and protested the support of armenia it's ummah problems not arab problems",AskMiddleEast,2024-04-06 18:18:33,1bxf7l2,Practical_Ant_4617,0,1,0,True,2024-04-01 04:03:56,0.0,0.0,7.0,109.0,116.0,84,NaN,Turkish police brutally arresting citizens pro...,0.91,84,0,2024-04-06 16:09:00
kyctg9w,1,Fuck off cleanshirt,CrazyFuckingVideos,2024-04-06 18:18:25,1bx9u9o,donkey_hat,0,1,0,True,2013-04-19 05:51:23,0.0,0.0,628.0,8705.0,9333.0,420,NaN,Feral kids terrorise security guard in UK and ...,0.94,420,0,2024-04-06 12:00:48
kyctfmu,1,[https://www.onondaganation.org/culture/wampum/two-row-wampum-belt-guswenta/](https://www.onondaganation.org/culture/wampum/two-row-wampum-belt-guswenta/) \n\n\nThis is why they should care,IsraelPalestine,2024-04-06 18:18:19,1bvmz1t,ZhaawGwa,0,1,0,True,2024-02-15 22:27:15,0.0,0.0,1.0,147.0,148.0,0,Now i am sure im about to be attacked for sayi...,Why should israel care,0.34,0,0,2024-04-04 13:20:56


In [ ]:
reddit_data.isnull().sum()